# EDA

Preprocessed Dataset

In [10]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import Utils as U

In [24]:
# Load aggregated eICU patient table

paths = U.load_paths()

data_dir = paths['data_dir']
flat = pd.read_csv(data_dir + 'preprocessed_flat.csv')

time_series = pd.read_csv(data_dir + 'preprocessed_timeseries.csv')

outcome = pd.read_csv(data_dir + 'raw_agg_patient_table.csv')[['patientunitstayid', 'died']]

flat.head()

ts_data = pd.merge(time_series, outcome, how = 'left', left_on = 'patient', right_on = 'patientunitstayid').dropna()

ts_data.to_csv(data_dir + "timeseries_readyformodel.csv", index= False)

In [15]:
outcome['patientunitstayid'] = outcome['patientunitstayid'].astype(int)

In [3]:
time_series.head()

patient  time  FiO2  bedside glucose  cvp  heartrate  noninvasivediastolic  \
0   141168     1   0.0              0.0  0.0   0.694915              0.000000   
1   141168     2   0.0              0.0  0.0   0.668079              0.244681   
2   141168     3   0.0              0.0  0.0   0.644068              0.244681   
3   141168     4   0.0              0.0  0.0   0.634181              0.244681   
4   141168     5   0.0              0.0  0.0   0.641243              0.244681   

   noninvasivemean  noninvasivesystolic  respiration  ...  respiration_mask  \
0         0.000000             0.000000          0.0  ...               0.0   
1         0.234513             0.156442          0.0  ...               0.0   
2         0.234513             0.156442          0.0  ...               0.0   
3         0.234513             0.156442          0.0  ...               0.0   
4         0.212389             0.156442          0.0  ...               0.0   

   sao2_mask  st1_mask  st2_mask  st3_mask  systemicdiastolic_mask  \
0       1.00       0.0       0.0       0.0                     0.0   
1       0.75       0.0       0.0       0.0                     0.0   
2       1.00       0.0       0.0       0.0                     0.0   
3       1.00       0.0       0.0       0.0                     0.0   
4       1.00       0.0       0.0       0.0                     0.0   

   systemicmean_mask  systemicsystolic_mask  temperature_mask      hour  
0                0.0                    0.0               0.0  0.695652  
1                0.0                    0.0               0.0  0.739130  
2                0.0                    0.0               0.0  0.782609  
3                0.0                    0.0               0.0  0.826087  
4                0.0                    0.0               0.0  0.869565  

[5 rows x 35 columns]

In [16]:
data = pd.merge(time_series, flat, how = 'left', left_on = 'patient', right_on = 'patient')

In [17]:
data = pd.merge(data, outcome[['patientunitstayid', 'died']], how = 'left', left_on = 'patient', right_on = 'patientunitstayid')

In [13]:
drop_cols = ['']

data.to_csv(data_dir + "preprocessed_total.csv")

In [18]:
test = data.dropna()



In [23]:
test.died.describe()

test.to_csv(data_dir + 'clean_preprocessed_total.csv', index= False)

### Distribution of numeric variables

In [5]:
patient_num_vars = patient.select_dtypes(include=np.number)
num_vars_to_exclude = ['patientunitstayid', 'patienthealthsystemstayid', 
                       'wardid']
num_cols_to_keep = [col for col in patient_num_vars.columns if 
                    col not in(num_vars_to_exclude)]
patient_num_vars = patient_num_vars.loc[:, num_cols_to_keep]

In [6]:
pd.set_option('display.max_columns', None)
patient_num_vars.describe()

hospitalid  admissionheight  admissionweight  dischargeweight  \
count  17167.000000     16914.000000     16777.000000      9991.000000   
mean     276.767170       168.438364        84.759627        84.257910   
std      116.124939        13.970985        31.324096        30.223758   
min       56.000000         0.000000         0.000000         0.000000   
25%      183.000000       160.000000        64.400000        64.200000   
50%      264.000000       168.000000        79.380000        79.800000   
75%      400.000000       177.800000        99.000000        99.000000   
max      459.000000       600.000000       838.000000       340.000000   

        icuduration  weightdiffafterdischarge         urine           wbc  \
count  17167.000000               9708.000000  16656.000000  16656.000000   
mean       8.816506                 -1.121805    862.797567      8.805526   
std       10.046396                 16.477694   1315.436835      8.754981   
min        0.000694               -756.200000     -1.000000     -1.000000   
25%        3.656597                 -2.220000     -1.000000     -1.000000   
50%        6.289583                  0.000000    113.745600      8.500000   
75%       10.804861                  2.400000   1357.689600     13.400000   
max      370.265278                139.700000  17033.414400    163.700000   

        temperature  respiratoryrate        sodium     heartrate  \
count  16656.000000     16656.000000  16656.000000  16656.000000   
mean      34.395799        28.230259    106.512992    104.538965   
std        8.693059        14.707892     58.204964     29.508599   
min       -1.000000        -1.000000     -1.000000     -1.000000   
25%       36.100000        13.000000    128.000000     94.000000   
50%       36.500000        30.000000    136.000000    108.000000   
75%       36.770000        38.000000    140.000000    122.000000   
max       42.300000        60.000000    173.000000    220.000000   

             meanbp            ph    creatinine       albumin       glucose  \
count  16656.000000  16656.000000  16656.000000  16656.000000  16656.000000   
mean      86.170347      1.791024      0.809288      0.423781    141.955367   
std       41.112144      3.942301      1.559317      1.901715     95.672316   
min       -1.000000     -1.000000     -1.000000     -1.000000     -1.000000   
25%       54.000000     -1.000000      0.400000     -1.000000     90.000000   
50%       67.000000     -1.000000      0.770000     -1.000000    132.000000   
75%      122.000000      7.301000      1.220000      2.600000    189.000000   
max      200.000000      7.715000     19.500000      7.400000   1237.000000   

          bilirubin          fio2          pao2          pco2           bun  \
count  16656.000000  16656.000000  16656.000000  16656.000000  16656.000000   
mean      -0.366978     19.273943     38.819975     15.604623     19.553122   
std        1.332129     32.238220     73.089378     25.321896     20.195906   
min       -1.000000     -1.000000     -1.000000     -1.000000     -1.000000   
25%       -1.000000     -1.000000     -1.000000     -1.000000      6.000000   
50%       -1.000000     -1.000000     -1.000000     -1.000000     15.000000   
75%        0.400000     40.000000     70.000000     37.825000     27.000000   
max       39.870000    100.000000    636.000000    147.300000    172.000000   

       meanapachescore  meanpredictedicumortality  meanpredictediculos  \
count     14611.000000               14611.000000         14611.000000   
mean         55.636301                   0.061843             4.289532   
std          25.173709                   0.212967             2.192518   
min          -1.000000                  -1.000000            -1.000000   
25%          38.000000                   0.013451             2.594999   
50%          52.000000                   0.034765             4.148490   
75%          69.000000                   0.097153             5.976981   
max         202.000

There are several variables related to the APACHE score that have a minimum value of -1. We assume that these values are missing so we will recode these as null.

In [7]:
cols_to_recode = ['urine', 'wbc', 'temperature', 'respiratoryrate', 'sodium',
                  'heartrate', 'meanbp', 'ph', 'creatinine', 'albumin', 
                  'glucose', 'bilirubin', 'fio2', 'pao2', 'pco2', 'bun', 
                  'meanapachescore', 'meanpredictedicumortality', 
                  'meanpredictediculos']

for col in cols_to_recode: 
    patient.loc[patient[col] == -1, col] = np.nan

### Distribution of categorical features

In [8]:
# Define function to get categorical percentages for categorical variables
def get_categorical_percentages(df):
    """
    This function computes the percentages of categories for each categorical
    variable in the data.
    
    :param df: data
    :type df: pandas dataframe
    
    :return: prints each variable with their categories and respective 
    proportions
    """
    cat_df = df.select_dtypes(include='category')
    for var in cat_df.columns:
        perc = df[var].value_counts(normalize=True)
        print(var)
        print(perc)

In [9]:
# Recode age, teaching status and gender variables
patient.age = patient.age.cat.rename_categories({'> 89': 90})
patient.age = pd.to_numeric(patient.age, errors='coerce')

patient['age0-9'] = np.nan
patient.loc[(patient.age >= 0) & (patient.age <= 9), 'age0-9'] = 1
patient.loc[(patient.age > 9), 'age0-9'] = 0
patient['age0-9'] = patient['age0-9'].astype('category')

patient['age10-19'] = np.nan
patient.loc[(patient.age >= 10) & (patient.age <= 19), 'age10-19'] = 1
patient.loc[(patient.age > 19), 'age10-19'] = 0
patient['age10-19'] = patient['age10-19'].astype('category')

patient['age20-29'] = np.nan
patient.loc[(patient.age >= 20) & (patient.age <= 29), 'age20-29'] = 1
patient.loc[(patient.age > 29), 'age20-29'] = 0
patient['age20-29'] = patient['age20-29'].astype('category')

patient['age30-39'] = np.nan
patient.loc[(patient.age >= 30) & (patient.age <= 39), 'age30-39'] = 1
patient.loc[(patient.age > 39), 'age30-39'] = 0
patient['age30-39'] = patient['age30-39'].astype('category')

patient['age40-49'] = np.nan
patient.loc[(patient.age >= 40) & (patient.age <= 49), 'age40-49'] = 1
patient.loc[(patient.age > 49), 'age40-49'] = 0
patient['age40-49'] = patient['age40-49'].astype('category')

patient['age50-59'] = np.nan
patient.loc[(patient.age >= 50) & (patient.age <= 59), 'age50-59'] = 1
patient.loc[(patient.age > 59), 'age50-59'] = 0
patient['age50-59'] = patient['age50-59'].astype('category')

patient['age60-69'] = np.nan
patient.loc[(patient.age >= 60) & (patient.age <= 69), 'age60-69'] = 1
patient.loc[(patient.age > 69), 'age60-69'] = 0
patient['age60-69'] = patient['age60-69'].astype('category')

patient['age70-79'] = np.nan
patient.loc[(patient.age >= 70) & (patient.age <= 79), 'age70-79'] = 1
patient.loc[(patient.age > 79), 'age70-79'] = 0
patient['age70-79'] = patient['age70-79'].astype('category')

patient['age80-89'] = np.nan
patient.loc[(patient.age >= 80) & (patient.age <= 89), 'age80-89'] = 1
patient.loc[(patient.age > 89), 'age80-89'] = 0
patient['age80-89'] = patient['age80-89'].astype('category')

patient['age>89'] = np.nan
patient.loc[(patient.age > 89), 'age>89'] = 1
patient.loc[(patient.age <= 89), 'age>89'] = 0
patient['age>89'] = patient['age>89'].astype('category')
patient.drop(columns='age', inplace=True)

patient.teachingstatus = np.where(patient.teachingstatus == 't', 1, 0)
patient.teachingstatus = patient.teachingstatus.astype('category')

patient['gendermale'] = np.nan
patient.loc[patient.gender == 'Male', 'gendermale'] = 1
patient.loc[patient.gender == 'Female', 'gendermale'] = 0
patient.gendermale = patient.gendermale.astype('category')
patient.drop(columns='gender', inplace=True)

In [10]:
get_categorical_percentages(patient)

ethnicity
Caucasian           0.779845
African American    0.120385
Other/Unknown       0.041645
Hispanic            0.037096
Asian               0.015122
Native American     0.005907
Name: ethnicity, dtype: float64
apacheadmissiondx
Pneumonia, bacterial                                                                       0.165958
Emphysema/bronchitis                                                                       0.147492
Arrest, respiratory (without cardiac arrest)                                               0.097629
Embolus, pulmonary                                                                         0.081202
Respiratory - medical, other                                                               0.077824
Obstruction-airway (i.e., acute epiglottitis, post-extubation edema, foreign body, etc)    0.042582
Pneumonia, other                                                                           0.039553
Pneumonia, aspiration                                             

There are a few features with highly skewed distributions towards one category over the other (99% vs 1%) including `aids` and `lymphoma`. We will drop these features,`apacheadmissiondx` given that there are a lot of categories for this feature, and `unitstaytype` since it is similar to the `hospitaladmitsource` variable.

In [11]:
patient.drop(columns=['aids', 'lymphoma', 'unitstaytype'], inplace=True)

In [12]:
# One hot encode categorical variables
categorical_df = patient.select_dtypes(include='category')
cols_to_encode = [col for col in categorical_df.columns 
                  if categorical_df[col].nunique() > 2]
ind_vars = pd.get_dummies(categorical_df.loc[:, cols_to_encode])
ind_vars = ind_vars.apply(lambda x: x.astype('category'))

# Remove hospital region from list of columns to drop because it will be used 
# for stratified sampling later
cols_to_encode.remove('hospitalregion')

patient = patient.drop(columns=cols_to_encode)
patient = pd.concat([patient, ind_vars], axis=1)

### Visualize numeric variables by target

In [13]:
num_df = patient.select_dtypes(include=np.number)
num_vars_to_exclude = ['patientunitstayid', 'patienthealthsystemstayid', 
                       'wardid', 'hospitalid']
num_cols_to_keep = [col for col in num_df.columns if 
                    col not in(num_vars_to_exclude)]
num_df = num_df.loc[:, num_cols_to_keep]
num_df['died'] = patient['died']

fig=plt.figure(figsize=(15,70))
variables = num_df.columns.drop('died')
for i, var in enumerate(variables):
    ax=fig.add_subplot(35, 4, i+1)
    ax=plt.gca()
    num_df.boxplot(column=var,
                   by='died', 
                   grid=False, 
                   ax=ax)
    ax.set(ylabel=var)
    ax.set_title(var)
fig.suptitle('')
fig.tight_layout()
plt.show()


### Distribution of categorical variables by target

In [14]:
# Get proportions of target for each categorical variable
categorical_df = patient.select_dtypes(include='category')
variables = categorical_df.columns.drop('died')  
for var in variables:
    print(pd.crosstab(categorical_df[var], 
                      categorical_df['died'], 
                      normalize='columns'), "\n")

died            0.0       1.0
intubated                    
0.0        0.836916  0.671366
1.0        0.163084  0.328634 

died       0.0       1.0
vent                    
0.0   0.622036  0.418571
1.0   0.377964  0.581429 

died           0.0       1.0
dialysis                    
0.0       0.973929  0.976179
1.0       0.026071  0.023821 

died                    0.0       1.0
immunosuppression                    
0.0                0.962141  0.939232
1.0                0.037859  0.060768 

died           0.0       1.0
diabetes                    
0.0       0.807447  0.824016
1.0       0.192553  0.175984 

died                 0.0      1.0
teachingstatus                   
0               0.751829  0.73659
1               0.248171  0.26341 

died                 0.0       1.0
hospitalregion                    
Midwest         0.341855  0.261965
Northeast       0.093376  0.109824
South           0.369996  0.446851
West            0.194773  0.181360 

died         0.0  1.0
age0-9        

There seems to be a significant difference in the proportion of patients who died between the categories of all categorical variables except gender and patients age 70-79.

### Mortality by discharge time

In [15]:
patient.hospitaldischargetime24 = pd.to_datetime(patient.hospitaladmittime24, format='%H:%M:%S')
patient['hospitaldischargetimehr'] = patient.hospitaldischargetime24.dt.hour

In [16]:
temp = patient.copy()
temp['died'] = pd.to_numeric(temp['died'], errors='coerce')

discharge_hour_mortality = (
    temp.groupby('hospitaldischargetimehr')
        .agg(total=pd.NamedAgg(column='died', aggfunc='count'), 
             mortalitycount=pd.NamedAgg(column='died', aggfunc='sum'))
)
discharge_hour_mortality = (
    discharge_hour_mortality.assign(
        mortalityrate=(discharge_hour_mortality['mortalitycount'] / 
                       discharge_hour_mortality['total'])
    )
)
discharge_hour_mortality = discharge_hour_mortality.reset_index()

In [17]:
fig = plt.figure(figsize = (8, 5))
plt.bar(discharge_hour_mortality.hospitaldischargetimehr, 
        discharge_hour_mortality.mortalityrate)
plt.xlabel('Hospital Discharge Hour')
plt.ylabel('Mortality Rate')
plt.title('Mortality Rate by Discharge Hour')
plt.show()

In [18]:
# Create indicator variable that indicates the timespan covering the drop
# in mortality rate from hours 8 to 14.
patient['dischargehour8-14'] = (
    np.where((patient.hospitaldischargetimehr >= 8) & 
             (patient.hospitaldischargetimehr <= 14), 1, 0)
)

patient.drop(columns=['hospitaladmittime24', 'hospitaldischargetime24', 
                      'unitadmittime24', 'unitdischargetime24', 
                      'hospitaldischargetimehr'], inplace=True)

### Check for missing values

We will exclude any columns with more than 50% missing values and impute the other columns that do not exceed this threshold. However, we will first remove all patients who have a missing discharge status.

In [19]:
patient.died.isna().sum()

178

In [20]:
patient.dropna(subset='died', inplace=True)
patient.shape

(16989, 258)

In [21]:
missing_val_count = patient.isna().sum()
missing_df = pd.DataFrame(data={'variable': patient.columns, 
                                'missing_val_count': missing_val_count.values})
missing_df['missing_val%'] = (
    100 * (missing_df['missing_val_count'] / patient.shape[0])
)

# Get columns with more than 80% missing data
cols_to_drop = (
    missing_df.loc[missing_df['missing_val%'] >= 80, 'variable'].tolist()
)

patient.drop(columns=cols_to_drop, inplace=True)

We will impute the numeric variables with the median and categorical variable with the most frequent label. Before doing so, we will first split the data into training and testing to avoid data leakage.

### Train-Test Split

We will randomize our train and test sets by patient and use stratified sampling by `hospitalregion` and `teachingstatus` to ensure that the distribution of patients from different regions and whether the hospital they are in is a teaching hospital are equivalent between the training and test data sets.

In [33]:
# Drop missing values by variables that will be used for stratified sampling
patient.dropna(subset=['hospitalregion', 'teachingstatus'], inplace=True)
print(patient.shape)
print(patient.uniquepid.nunique())

# Remove irrelevant columns
patient.drop(columns=['patientunitstayid', 'patienthealthsystemstayid', 
                      'wardid', 'uniquepid'], 
             inplace=True)


X = patient.drop(columns=['died', 'hospitalid'])
y = patient.died
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    stratify=X[['hospitalregion',
                                                                'teachingstatus']],
                                                    random_state=607)

print(X_train.shape)
print(X_test.shape)

(15950, 211)
15950
(12760, 205)
(3190, 205)


##### Sanity check for stratified sampling

In [34]:
print(X_train.teachingstatus.value_counts(normalize=True))
print(X_test.teachingstatus.value_counts(normalize=True))

0    0.733699
1    0.266301
Name: teachingstatus, dtype: float64
0    0.733856
1    0.266144
Name: teachingstatus, dtype: float64


In [35]:
print(X_train.hospitalregion.value_counts(normalize=True))
print(X_test.hospitalregion.value_counts(normalize=True))

# Drop hospital region column 
X_train.drop(columns='hospitalregion', inplace=True)
X_test.drop(columns='hospitalregion', inplace=True)

South        0.379624
Midwest      0.331897
West         0.193025
Northeast    0.095455
Name: hospitalregion, dtype: float64
South        0.379310
Midwest      0.331975
West         0.193417
Northeast    0.095298
Name: hospitalregion, dtype: float64


### Impute missing values

In [36]:
# Impute numeric variables with median
numeric_vars = X_train.select_dtypes(include=np.number).columns
imputer = SimpleImputer(strategy='median')
imputer = imputer.fit(X_train[numeric_vars])
X_train[numeric_vars] = imputer.transform(X_train[numeric_vars])
X_test[numeric_vars] = imputer.transform(X_test[numeric_vars])

In [37]:
# Impute categorical variables with most frequent label
categorical_vars = X_train.select_dtypes(include='category').columns
imputer = SimpleImputer(strategy='most_frequent')
imputer = imputer.fit(X_train[categorical_vars])
X_train[categorical_vars] = imputer.transform(X_train[categorical_vars])
X_test[categorical_vars] = imputer.transform(X_test[categorical_vars])

In [38]:
X_train.isna().sum().sum()

0

In [39]:
X_test.isna().sum().sum()

0

In [40]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

print(train.shape)
print(test.shape)

# Save training and test datasets to csv 
train.to_csv(data_dir + 'train.csv', index=False)
test.to_csv(data_dir + 'test.csv', index=False)

(12760, 205)
(3190, 205)
